<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Sarah", "transactions": [{"transaction-id": 6861, "amount": 206}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Sarah", "transactions": [{"transaction-id": 6861, "amount": 206}]}\n',
 '{"id": 1, "name": "Jerry", "transactions": [{"transaction-id": 743, "amount": 796}, {"transaction-id": 2081, "amount": 761}, {"transaction-id": 3492, "amount": 859}, {"transaction-id": 5980, "amount": 885}, {"transaction-id": 6285, "amount": 862}, {"transaction-id": 8658, "amount": 782}]}\n',
 '{"id": 2, "name": "Norbert", "transactions": [{"transaction-id": 40, "amount": 118}, {"transaction-id": 214, "amount": 92}, {"transaction-id": 1280, "amount": 112}, {"transaction-id": 2108, "amount": 111}, {"transaction-id": 2595, "amount": 103}, {"transaction-id": 2791, "amount": 101}, {"transaction-id": 2877, "amount": 87}, {"transaction-id": 3188, "amount": 124}, {"transaction-id": 3544, "amount": 108}, {"transaction-id": 3689, "amount": 106}, {"transaction-id": 3964, "amount": 115}, {"transaction-id": 4347, "amount": 123}, {"transaction-id": 4616, "amount": 109}, {"transaction-id": 5228, "amount": 8

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Sarah',
  'transactions': [{'transaction-id': 6861, 'amount': 206}]},
 {'id': 1,
  'name': 'Jerry',
  'transactions': [{'transaction-id': 743, 'amount': 796},
   {'transaction-id': 2081, 'amount': 761},
   {'transaction-id': 3492, 'amount': 859},
   {'transaction-id': 5980, 'amount': 885},
   {'transaction-id': 6285, 'amount': 862},
   {'transaction-id': 8658, 'amount': 782}]},
 {'id': 2,
  'name': 'Norbert',
  'transactions': [{'transaction-id': 40, 'amount': 118},
   {'transaction-id': 214, 'amount': 92},
   {'transaction-id': 1280, 'amount': 112},
   {'transaction-id': 2108, 'amount': 111},
   {'transaction-id': 2595, 'amount': 103},
   {'transaction-id': 2791, 'amount': 101},
   {'transaction-id': 2877, 'amount': 87},
   {'transaction-id': 3188, 'amount': 124},
   {'transaction-id': 3544, 'amount': 108},
   {'transaction-id': 3689, 'amount': 106},
   {'transaction-id': 3964, 'amount': 115},
   {'transaction-id': 4347, 'amount': 123},
   {'transaction-id': 4616

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 3,
  'name': 'Alice',
  'transactions': [{'transaction-id': 790, 'amount': 1063},
   {'transaction-id': 4060, 'amount': 1103},
   {'transaction-id': 4577, 'amount': 1077},
   {'transaction-id': 6531, 'amount': 1166},
   {'transaction-id': 6907, 'amount': 1126},
   {'transaction-id': 7034, 'amount': 1087},
   {'transaction-id': 9459, 'amount': 1011}]},
 {'id': 18,
  'name': 'Alice',
  'transactions': [{'transaction-id': 91, 'amount': 93},
   {'transaction-id': 220, 'amount': 178},
   {'transaction-id': 261, 'amount': 65},
   {'transaction-id': 265, 'amount': 176},
   {'transaction-id': 374, 'amount': 175},
   {'transaction-id': 629, 'amount': 288},
   {'transaction-id': 781, 'amount': 137},
   {'transaction-id': 873, 'amount': 186},
   {'transaction-id': 992, 'amount': 194},
   {'transaction-id': 1032, 'amount': 242},
   {'transaction-id': 1879, 'amount': 217},
   {'transaction-id': 2017, 'amount': 155},
   {'transaction-id': 2387, 'amount': 231},
   {'transaction-id': 2460, 'am

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 7},
 {'name': 'Alice', 'count': 33},
 {'name': 'Alice', 'count': 23},
 {'name': 'Alice', 'count': 28},
 {'name': 'Alice', 'count': 10})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(7, 33, 23, 28, 10)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

34.586666666666666

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 790, 'amount': 1063},
  {'transaction-id': 4060, 'amount': 1103},
  {'transaction-id': 4577, 'amount': 1077},
  {'transaction-id': 6531, 'amount': 1166},
  {'transaction-id': 6907, 'amount': 1126},
  {'transaction-id': 7034, 'amount': 1087},
  {'transaction-id': 9459, 'amount': 1011}],
 [{'transaction-id': 91, 'amount': 93},
  {'transaction-id': 220, 'amount': 178},
  {'transaction-id': 261, 'amount': 65},
  {'transaction-id': 265, 'amount': 176},
  {'transaction-id': 374, 'amount': 175},
  {'transaction-id': 629, 'amount': 288},
  {'transaction-id': 781, 'amount': 137},
  {'transaction-id': 873, 'amount': 186},
  {'transaction-id': 992, 'amount': 194},
  {'transaction-id': 1032, 'amount': 242},
  {'transaction-id': 1879, 'amount': 217},
  {'transaction-id': 2017, 'amount': 155},
  {'transaction-id': 2387, 'amount': 231},
  {'transaction-id': 2460, 'amount': 117},
  {'transaction-id': 2621, 'amount': 247},
  {'transaction-id': 2759, 'amount': 188},
  {'transaction-

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 790, 'amount': 1063},
 {'transaction-id': 4060, 'amount': 1103},
 {'transaction-id': 4577, 'amount': 1077})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(1063, 1103, 1077)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

631.7219545104086

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 72), ('Alice', 72), ('Alice', 78), ('Alice', 78), ('Bob', 131), ('Bob', 131), ('Bob', 141), ('Bob', 143), ('Charlie', 96), ('Charlie', 96), ('Charlie', 104), ('Charlie', 104), ('Dan', 36), ('Dan', 36), ('Dan', 39), ('Dan', 39), ('Edith', 133), ('Edith', 135), ('Edith', 145), ('Edith', 146), ('Frank', 135), ('Frank', 137), ('Frank', 146), ('Frank', 149), ('George', 131), ('George', 131), ('George', 142), ('George', 142), ('Hannah', 80), ('Hannah', 81), ('Hannah', 87), ('Hannah', 88), ('Ingrid', 60), ('Ingrid', 60), ('Ingrid', 65), ('Ingrid', 65), ('Jerry', 104), ('Jerry', 104), ('Jerry', 192), ('Kevin', 192), ('Kevin', 192), ('Kevin', 208), ('Kevin', 208), ('Laura', 137), ('Laura', 137), ('Laura', 144), ('Laura', 144), ('Michael', 198), ('Michael', 202), ('Michael', 213), ('Michael', 219), ('Norbert', 126), ('Norbert', 130), ('Norbert', 142), ('Norbert', 143), ('Oliver', 72), ('Oliver', 72), ('Oliver', 78), ('Oliver', 78), ('Patricia', 84), ('Patricia', 84), ('Patricia', 91),

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 300), ('Bob', 546), ('Charlie', 400), ('Dan', 150), ('Edith', 559), ('Frank', 567), ('George', 546), ('Hannah', 336), ('Ingrid', 250), ('Jerry', 400), ('Kevin', 800), ('Laura', 562), ('Michael', 832), ('Norbert', 541), ('Oliver', 300), ('Patricia', 350), ('Quinn', 400), ('Ray', 493), ('Sarah', 624), ('Tim', 634), ('Ursula', 250), ('Victor', 646), ('Wendy', 674), ('Xavier', 479), ('Yvonne', 235), ('Zelda', 350)]
CPU times: user 146 ms, sys: 20.4 ms, total: 167 ms
Wall time: 563 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


,id,name,transactions
0,0,Sarah,"[{'transaction-id': 6861, 'amount': 206}]"
1,1,Jerry,"[{'transaction-id': 743, 'amount': 796}, {'tra..."
2,2,Norbert,"[{'transaction-id': 40, 'amount': 118}, {'tran..."
3,3,Alice,"[{'transaction-id': 790, 'amount': 1063}, {'tr..."
4,4,Kevin,"[{'transaction-id': 111, 'amount': 1666}, {'tr..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 317 ms, sys: 18.4 ms, total: 336 ms
Wall time: 1.77 s


name
Alice      300
Bob        546
Charlie    400
Dan        150
Edith      559
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Sarah', 'amount': 206, 'transaction-id': 6861},
 {'id': 1, 'name': 'Jerry', 'amount': 796, 'transaction-id': 743},
 {'id': 1, 'name': 'Jerry', 'amount': 761, 'transaction-id': 2081})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Sarah,206,6861
1,1,Jerry,796,743
2,1,Jerry,761,2081
3,1,Jerry,859,3492
4,1,Jerry,885,5980


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 245 ms, sys: 28.6 ms, total: 273 ms
Wall time: 1.4 s


name
Alice       10376
Bob         23426
Charlie      9919
Dan         11284
Edith       28034
Frank       26033
George      18396
Hannah      19748
Ingrid       4093
Jerry       18032
Kevin       27374
Laura       17074
Michael     23899
Norbert     19588
Oliver      14376
Patricia     8771
Quinn       18084
Ray         16059
Sarah       29550
Tim         23012
Ursula      21738
Victor      25341
Wendy       45305
Xavier      20210
Yvonne       9298
Zelda       10980
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()

/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/mi